In [3]:
import pl from "npm:nodejs-polars";
import { display } from "https://deno.land/x/display@v0.1.1/mod.ts";

let response = await fetch(
"https://gist.githubusercontent.com/agustinustheo/195f32a4a6c68c493056c883d959ca35/raw/c7363d8b916ab00a2d1747adb89fca120da29f42/mock_financial_data.csv",
);

let categories = ['Salaries', 'R&D', 'Marketing', 'Utilities', 'Rent', 'Equipment', 'Software', 'Hardware', 'Consulting', 'Office Supplies'];

let data = await response.text();

let df = pl.readCSV(data, { sep: "," });

await display(df.sample(10));

In [4]:
import * as d3 from "npm:d3";
import { createCanvas } from "https://deno.land/x/skia_canvas/mod.ts";

// Sample data
const sampleData1 = [];
for(let i = 0; i < categories.length; i++) {
    const category = categories[i];
    const lastDataPoint = df.tail(1).toRecords()[0];
    sampleData1.push({
        category,
        amount: lastDataPoint[category],
    });
}

// Set the dimensions of the canvas
const width = 500;
const height = 500;
const radius = Math.min(width, height) / 2;

// Create a pie function
const pie = d3.pie().value(d => d.amount);

// Create an arc generator for the slices
const arc = d3.arc()
    .innerRadius(0)
    .outerRadius(radius);

// Create an arc generator for the labels
const labelArc = d3.arc()
    .innerRadius(radius - 40) // Adjust to position the labels
    .outerRadius(radius - 40);

// Create the canvas
const canvas = createCanvas(width, height);
const ctx = canvas.getContext("2d");

// Translate to center the pie chart
ctx.translate(width / 2, height / 2);

// Draw the pie chart
pie(sampleData1).forEach((d, i) => {
    // Draw the slice
    ctx.beginPath();
    arc.context(ctx)(d);
    ctx.fillStyle = d3.schemeCategory10[i % 10];
    ctx.fill();

    // Draw the label
    ctx.fillStyle = "#000"; // Label color
    ctx.textAlign = "center";
    ctx.textBaseline = "middle";
    
    const centroid = labelArc.centroid(d);
    ctx.fillText(d.data.category, centroid[0], centroid[1]);
});

// Display the canvas
await display(canvas);

In [ ]:
import * as Plot from "npm:@observablehq/plot";
import {
    DOMParser,
    SVGElement
} from "npm:linkedom";
const document = new DOMParser().parseFromString(
    `<!DOCTYPE html><html lang="en"></html>`,
    "text/html",
);

let records = df.toRecords();

let sampleData2 = [];
for(let i = 0; i < records.length; i++) {
    const currentRecord = records[i];
    for(let x = 0; x < categories.length; x++) {
        const currentCategory = categories[x];
        sampleData2.push({
            date: currentRecord["Date"],
            category: currentCategory;
            amount: currentRecord[currentCategory]
        })
    }
}

await display(
    Plot.barY(
        sampleData2,
        Plot.groupX({
            y: "amount"
        }, {
            x: "date",
            fill: "category"
        })
    ).plot({
        color: {
            legend: true
        },
        document
    })
);